## Dataset

Polarity Dataset. Pang/Lee ACL 2004

http://www.cs.cornell.edu/people/pabo/movie-review-data/

http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

## Data Directories

In [1]:
pos_path = "./review_polarity/txt_sentoken/pos"
neg_path = "./review_polarity/txt_sentoken/neg"

## Load data

In [2]:
import os

def read_text_files(category_path):
    file_list = os.listdir(category_path)
    texts = []

    for fname in file_list:
        with open(os.path.join(category_path, fname), "r") as f:
            lines = f.readlines()
            texts.extend(lines)
    
    return texts

In [3]:
pos_texts = read_text_files(pos_path)
neg_texts = read_text_files(neg_path)

## Clean extra characters
new lines, tabs yada yada

In [4]:
def clean(texts):
    cleaned = []
    for text in texts:
        cleaned_text = text.strip().lower()
        cleaned.append(cleaned_text)

    return cleaned

In [5]:
pos_cleaned = clean(pos_texts)
neg_cleaned = clean(neg_texts)

## Embedding - Word2Vec

In [33]:
from nltk.tokenize import word_tokenize as tokenize
from nltk.corpus import stopwords
from tqdm import tqdm


# remove stopwords, use the already cleaned lists
# then label

# label: 1 for pos, 0 for neg
def process_one_class(label, data):
    processed = list()
    for i in tqdm(range(len(data)), desc=f"preprocess_{label}"):
        text = data[i]
        raw = tokenize(text)
        cleaned = []
        
        for tok in raw:
            if tok not in ignore:
                cleaned.append(tok)
        

        processed.append((label, cleaned))
        
    return processed


def preprocess():
    data = [] # label, tokens
    
    pos_processed = process_one_class(1, pos_cleaned)
    neg_processed = process_one_class(0, neg_cleaned)
        
    data.extend(pos_processed)
    data.extend(neg_processed)
    
    return data

In [34]:
data = preprocess()

preprocess_0: 100%|██████████| 31783/31783 [00:04<00:00, 7348.64it/s]


In [37]:
sentences = [s[1] for s in data]
labels = [s[0] for s in data]

### Word2Vec from Gensim

In [59]:
import gensim
import multiprocessing
import random
from gensim.models import Word2Vec

In [51]:
# 300 dims for the embedding
EMBEDDING_DIMS = 300

In [52]:
cores = multiprocessing.cpu_count()

# keep one cpu core free or some operating systems may kill the process :P
model = Word2Vec(size=EMBEDDING_DIMS, workers=cores-1, max_vocab_size=100000)

# build vocab
model.build_vocab(sentences)

# train
%time model.train(sentences, total_examples=len(sentences), epochs=50)

CPU times: user 9min 29s, sys: 438 ms, total: 9min 29s
Wall time: 27 s


(34695961, 46480100)

### Vocabulary

In [70]:
vocabulary = model.wv.vocab
vocab_len = len(vocabulary.keys())

## Word2Vec Embedding to Matrix
Because we need one for CNN as embedding weights

In [79]:
import numpy as np

# n_rows : number of words
# n_cols : embedding dims
def create_embedding_matrix(vocabulary, model, n_rows, n_cols):
    matrix = np.zeros((n_rows, n_cols))
    
    for idx, word in enumerate(vocabulary.keys()):
        embedding_vector = model.wv[word]
        matrix[idx] = embedding_vector
        
    return matrix
        

embed_matrix = create_embedding_matrix(vocabulary, model, vocab_len, EMBEDDING_DIMS)

## Create input matrix for CNN
